In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/trainecza/Takmlarla Paylalacak Veri Seti.csv
/kaggle/input/eczacibasi-projesi/sample_submission.csv
/kaggle/input/eczacibasi-projesi/Takmlarla Paylalacak Veri Seti.xlsx
/kaggle/input/eczacibasi-projesi/id.xlsx
/kaggle/input/testecza/id.csv
/kaggle/input/enflasyon/Untitled 1.csv
/kaggle/input/eczacibasiiii/Takmlarla Paylalacak Veri Seti.csv
/kaggle/input/eczacibasiiii/id.csv


In [2]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from xgboost import plot_importance
import seaborn as sns
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import mean_squared_error,r2_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMRegressor
pd.set_option('display.max_columns', None)
from sklearn.linear_model import LinearRegression,PoissonRegressor

In [3]:
train = pd.read_csv("../input/trainecza/Takmlarla Paylalacak Veri Seti.csv")
test = pd.read_csv("../input/testecza/id.csv")
submission = pd.read_csv("/kaggle/input/eczacibasi-projesi/sample_submission.csv")

In [4]:
a = train.groupby(["customer","item","date"])["order"].sum()
a

customer  item  date   
A         0     2018-1     216.00
                2018-10    576.00
                2018-11    144.00
                2018-12     72.00
                2018-2     576.00
                            ...  
C         80    2020-1      36.92
                2020-2     221.52
                2020-3     295.36
                2020-6     184.60
                2020-7     110.76
Name: order, Length: 2382, dtype: float64

In [5]:
a = pd.merge(a, train, how="outer",on=["customer","item","date"])

In [6]:
a = a.drop_duplicates(["date","order_x"])

In [7]:
a = a.drop(["order_y"], axis=1).rename({"order_x":"order"} , axis=1)

In [8]:
train = a.copy()

In [9]:
train["Year"] = train.date.str[:4]
test["Year"]= test.date.str[:4]

train["Month"] = train.date.str[5:]
test["Month"] = test.date.str[5:]

In [10]:
train.Year = train.Year.astype("int")
train.Month = train.Month.astype("int")

test.Year = test.Year.astype("int")
test.Month = test.Month.astype("int")

#train = train.drop(["date"], axis=1)
#test = test.drop(["date"], axis=1)

In [11]:
train.head()

,customer,item,date,order,Year,Month
0,A,0,2018-1,216.0,2018,1
2,A,0,2018-10,576.0,2018,10
6,A,0,2018-11,144.0,2018,11
8,A,0,2018-12,72.0,2018,12
9,A,0,2018-2,576.0,2018,2


In [12]:
train.loc[train["Month"].isin([1,2,3]),"quarter"]=1
train.loc[train["Month"].isin([4,5,6]),"quarter"]=2
train.loc[train["Month"].isin([7,8,9]),"quarter"]=3
train.loc[train["Month"].isin([10,11,12]),"quarter"]=4

test.loc[test["Month"].isin([1,2,3]),"quarter"]=1
test.loc[test["Month"].isin([4,5,6]),"quarter"]=2
test.loc[test["Month"].isin([7,8,9]),"quarter"]=3
test.loc[test["Month"].isin([10,11,12]),"quarter"]=4

In [13]:
train.loc[(train["Month"]>=3)&(train["Year"]>=2020),"with_corona"]=1
train["with_corona"]=train["with_corona"].fillna(0)

test.loc[(test["Month"]>=3)&(test["Year"]>=2020),"with_corona"]=1
test["with_corona"]=test["with_corona"].fillna(0)

In [14]:
train.head()

,customer,item,date,order,Year,Month,quarter,with_corona
0,A,0,2018-1,216.0,2018,1,1.0,0.0
2,A,0,2018-10,576.0,2018,10,4.0,0.0
6,A,0,2018-11,144.0,2018,11,4.0,0.0
8,A,0,2018-12,72.0,2018,12,4.0,0.0
9,A,0,2018-2,576.0,2018,2,1.0,0.0


In [15]:
test.quarter.nunique()

3

In [16]:
agg=train.groupby(["quarter","item"])["order"].agg({"sum","mean"}).rename({"sum":"sum_item_order","mean":"mean_item_order"},axis=1)

train=pd.merge(train,agg,on=["quarter","item"])
test=pd.merge(test,agg,on=["quarter","item"])

In [17]:
train.head()

,customer,item,date,order,Year,Month,quarter,with_corona,sum_item_order,mean_item_order
0,A,0,2018-1,216.0,2018,1,1.0,0.0,11304.0,1256.0
1,A,0,2018-2,576.0,2018,2,1.0,0.0,11304.0,1256.0
2,A,0,2018-3,288.0,2018,3,1.0,0.0,11304.0,1256.0
3,A,0,2019-1,1008.0,2019,1,1.0,0.0,11304.0,1256.0
4,A,0,2019-2,1944.0,2019,2,1.0,0.0,11304.0,1256.0


# Feature engineering

In [18]:
agg = train.groupby(["customer","item"])["order"].agg({"min","mean","max","sum","count"})

train = pd.merge(train, agg, how="inner", on=["customer","item"])

In [19]:
test = pd.merge(test,agg,how="inner",on=["customer","item"])

In [20]:

train.shape, test.shape

((2062, 15), (498, 15))

In [21]:
train.head()

,customer,item,date,order,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum
0,A,0,2018-1,216.0,2018,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0
1,A,0,2018-2,576.0,2018,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0
2,A,0,2018-3,288.0,2018,3,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0
3,A,0,2019-1,1008.0,2019,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0
4,A,0,2019-2,1944.0,2019,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0


In [22]:
agg=train.groupby(["customer"])["order"].agg({"max","min","mean","count","sum"}).rename({"sum":"sum_cust","max":"max_cust","min":"min_cust",
                                                                                     "count":"count_cust","mean":"mean_cust"},axis=1)
train=pd.merge(train,agg,on=["customer"])
test=pd.merge(test,agg,on=["customer"])

In [23]:
test.head()

,id,customer,item,date,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum,max_cust,min_cust,mean_cust,count_cust,sum_cust
0,1,A,0,2020-10,2020,10,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64
1,2,A,0,2020-11,2020,11,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64
2,3,A,0,2020-12,2020,12,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64
3,4,A,0,2020-8,2020,8,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64
4,5,A,0,2020-9,2020,9,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64


In [24]:
train = train.drop(["date"], axis=1)
test = test.drop(["date"], axis=1)

train = pd.get_dummies(train)
test  = pd.get_dummies(test)


In [25]:
train.head()

,item,order,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum,max_cust,min_cust,mean_cust,count_cust,sum_cust,customer_A,customer_B,customer_C
0,0,216.0,2018,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
1,0,576.0,2018,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
2,0,288.0,2018,3,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
3,0,1008.0,2019,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
4,0,1944.0,2019,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0


In [26]:
X = train.drop(["order","sum","max","max_cust","mean","sum_item_order","quarter","sum_cust"], axis=1)
y = train["order"]

In [27]:
X.head()

,item,Year,Month,with_corona,mean_item_order,min,count,min_cust,mean_cust,count_cust,customer_A,customer_B,customer_C
0,0,2018,1,0.0,1256.0,72.0,30,25.0,1923.975809,1131,1,0,0
1,0,2018,2,0.0,1256.0,72.0,30,25.0,1923.975809,1131,1,0,0
2,0,2018,3,0.0,1256.0,72.0,30,25.0,1923.975809,1131,1,0,0
3,0,2019,1,0.0,1256.0,72.0,30,25.0,1923.975809,1131,1,0,0
4,0,2019,2,0.0,1256.0,72.0,30,25.0,1923.975809,1131,1,0,0


In [28]:
params={'max_depth':2,"n_estimators":10,"reg_lambda":100,"reg_alpha":0.9}
rmse = []
tscv = TimeSeriesSplit(gap=0 , n_splits=5)
r2s=[]
train_r2s=[]

for train_index, test_index in tscv.split(X):
    X_train_, X_test_ = X.iloc[train_index], X.iloc[test_index]
    y_train_, y_test_ = y.iloc[train_index], y.iloc[test_index]
    
    model=LGBMRegressor(**params)
    model.fit(X_train_, y_train_)
    y_pred = model.predict(X_test_)
    rmse_x = np.sqrt(mean_squared_error(y_test_, y_pred))
    r2=r2_score(y_test_,y_pred)
    train_r2=model.score(X_train_,y_train_)
    rmse.append(rmse_x)
    r2s.append(r2)
    train_r2s.append(train_r2)
    
    
print("rmse:",rmse)
print("test r2:",r2s)
print("train r2:",train_r2s)
print("rmse mean:",np.mean(rmse),
     "train r2 mean: ",np.mean(train_r2s),
     "test r2 mean: ",np.mean(r2s))

rmse: [2211.804152201231, 1486.3634573004679, 2938.2286627482595, 2104.632316464517, 1288.7669595156865]
test r2: [-0.01680262210024508, 0.34522916117119096, 0.17784774695071914, 0.39506049736126814, 0.19995324817625648]
train r2: [0.2677118075875353, 0.43884717143881435, 0.4179019359220568, 0.3769705210345753, 0.4180015412907704]
rmse mean: 2005.9591096460322 train r2 mean:  0.38388659545475046 test r2 mean:  0.22025760631183794


In [29]:
train.head()

,item,order,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum,max_cust,min_cust,mean_cust,count_cust,sum_cust,customer_A,customer_B,customer_C
0,0,216.0,2018,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
1,0,576.0,2018,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
2,0,288.0,2018,3,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
3,0,1008.0,2019,1,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
4,0,1944.0,2019,2,1.0,0.0,11304.0,1256.0,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0


In [30]:
test.head()

,id,item,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum,max_cust,min_cust,mean_cust,count_cust,sum_cust,customer_A,customer_B,customer_C
0,1,0,2020,10,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
1,2,0,2020,11,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
2,3,0,2020,12,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
3,4,0,2020,8,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
4,5,0,2020,9,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0


In [31]:
#X_train=train.drop(["order","sum","max","max_cust","mean","sum_item_order","quarter","sum_cust"],axis=1)
#X_test=test.drop(["id","sum","max","max_cust","mean","sum_item_order","quarter","sum_cust"],axis=1)

X_train=train.drop(["order"],axis=1)
X_test=test.drop(["id"],axis=1)
y_train=train["order"]

In [32]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [34]:
model=PoissonRegressor(max_iter=3000,alpha=10)
model_fitted=model.fit(X_train_scaled,y_train)
pred=model.predict(X_test_scaled)
print("Train Score:",model.score(X_train_scaled,y_train))

Train Score: 0.5760888152335877


In [35]:
pred[:10]

array([1222.83106894, 1186.47246201, 1151.19491061, 1346.25624641,
       1306.22782143, 1405.86379509, 1121.49673563, 1088.15111658,
       1055.79696747, 1164.3961015 ])

In [36]:
test.head()

,id,item,Year,Month,quarter,with_corona,sum_item_order,mean_item_order,max,min,mean,count,sum,max_cust,min_cust,mean_cust,count_cust,sum_cust,customer_A,customer_B,customer_C
0,1,0,2020,10,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
1,2,0,2020,11,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
2,3,0,2020,12,4.0,1.0,2736.0,456.000000,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
3,4,0,2020,8,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0
4,5,0,2020,9,3.0,1.0,8928.0,1275.428571,3453.0,72.0,1216.7,30,36501.0,13953.6,25.0,1923.975809,1131,2176016.64,1,0,0


In [37]:
submission.head()

,id,order
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0


In [38]:
submission["order"]=pred

In [39]:
submission_csv=submission.to_csv("submission.csv",index=False)
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv

In [41]:

#perm = PermutationImportance(model, random_state=0).fit(X_train,y_train)
#eli5.show_weights(perm, feature_names = X_train.columns.tolist(),top=50)


In [ ]:
#plt.figure(figsize=(20,7))


#sns.heatmap(X_train.corr(),annot=True,cmap="coolwarm");